# Presentación del Código y NAT Traversal

## Teoría de NAT Traversal

### ¿Qué es NAT?

- Network Address Translation (NAT) es una técnica que permite que varios dispositivos compartan una única dirección IP pública.

- Esto es común en redes domésticas y corporativas para preservar direcciones IP públicas.

### Desafío de NAT

- NAT puede dificultar la comunicación punto a punto debido a la ocultación de direcciones IP internas.

- Resolver este desafío es crucial para aplicaciones de voz sobre IP y otras comunicaciones en tiempo real.

### NAT Traversal

- NAT Traversal se refiere a las técnicas utilizadas para sortear las restricciones de NAT y establecer comunicaciones directas entre dispositivos detrás de NAT.

- Algunas técnicas comunes incluyen STUN, TURN y ICE.

## Código a realizar

 Determinar (y mostrar) el end-point externo (público) del dispositivo NAT cuando se ejecuta NAT_traversal.py, clasificar (y mostrar) la clasificación de tu dispositivo NAT.."

### Clasificación de NAT (fundamentos)

**Dependiendo de cómo se acepta (o no) el tráfico entrante, los NAT se clasifican de la siguiente manera:**

- EIF (Filtrado Independiente del Punto Final): cuando los paquetes entrantes se reenvían sin restricciones adicionales. Esto se utiliza en dispositivos NAT de tipo Full Cone.

- ARF (Filtrado con Restricción de Dirección): cuando solo se reenvían los paquetes entrantes que tienen la misma dirección IP externa que la registrada en el registro de Traducción de Tráfico (TT) correspondiente al punto final interno. Esta política se utiliza en dispositivos NAT de tipo (Address) Restringido.

- ERF (Filtrado con Restricción del Punto Final): cuando solo se reenvían los paquetes entrantes externos que tienen el mismo punto final externo que el registrado en el registro TT correspondiente al punto final interno. Esta política se utiliza en dispositivos NAT de tipo Port Restricted Cone (PRCN) y NAT Simétrico.

### `NAT_STUN()`

- Utiliza la biblioteca `stun` para obtener información sobre el tipo de NAT y la IP externa.
- Registra el puerto externo, la IP externa y el tipo de NAT para cada punto final.

In [2]:
def NAT_STUN():
    with open("lista_ip.txt") as archivo:
        for linea in archivo:
            ip = linea.strip()
            nat_type, external_ip, external_port = stun.get_ip_info(stun_host=ip)
            print(f"Puerto Externo: {external_port}, "
                  f"IP Externa: {external_ip}, "
                  f"End-Point: {external_ip}:{external_port}, "
                  f"Tipo de NAT: {nat_type}")
            endPoint = (external_ip, external_port)    
            if (type(external_ip)==int and type(external_port)==int):
                NAT_Traversal.endPoints.append(endPoint)
    archivo.close()

### `NAT_TYPE()`

- Compara los tipos de NAT de los puntos finales.
- Informa si el NAT es abierto (Full Cone) o simétrico.

In [4]:
def NAT_TYPE():
    if all(endPoint == NAT_Traversal.endPoints[0] for endPoint in NAT_Traversal.endPoints):
        print("NAT abierto (Full Cone)")
        print("Puede establecer conexion con otro dispositivo. Para ello ejecute el siguiente comando")
        print("python3 NAT-traversal -l \'puerto de escucha\' -a \'ip de destino\' -p \'puerto de destino\'")     
    else:
        print("NAT simetrico")
        print("No puede establecer conexion con otro dispositivo.")

### `NAT_traversal`

In [ ]:
#!/usr/bin/env python
# PYTHON_ARGCOMPLETE_OK

import minimal
import stun

class NAT_Traversal(minimal.Minimal):

    endPoints = []

    def NAT_STUN():
            with open("lista_ip.txt") as archivo:
                for linea in archivo:
                    ip = linea.strip()
                    nat_type, external_ip, external_port = stun.get_ip_info(stun_host=ip)
                    print(f"Puerto Externo: {external_port}, "
                          f"IP Externa: {external_ip}, "
                          f"End-Point: {external_ip}:{external_port}, "
                          f"Tipo de NAT: {nat_type}")
                    endPoint = (external_ip, external_port)
                    if (type(external_ip)==int and type(external_port)==int):
                        NAT_Traversal.endPoints.append(endPoint)
            archivo.close()
    
    def NAT_TYPE():
            if all(endPoint == NAT_Traversal.endPoints[0] for endPoint in NAT_Traversal.endPoints):
                print("NAT abierto (Full Cone)")
                print("Puede establecer conexion con otro dispositivo. Para ello ejecute el siguiente comando: 
                       python3 NAT-traversal -l \'puerto de escucha\' -a \'ip de destino\' -p \'puerto de destino\'")     
            else:
                print("NAT simetrico")
                print("No puede establecer conexion con otro dispositivo.")
    
    def __init__(self):
        
        ''' Constructor. Basically initializes the sockets stuff. '''
        super().__init__()

if __name__ == "__main__":
     
    minimal.args = minimal.parser.parse_args()

    if minimal.args.listening_port != 4444 and minimal.args.destination_address != "localhost" and minimal.args.destination_port != 4444:
        intercom = NAT_Traversal()
        try:
            intercom.run()
        except KeyboardInterrupt:
            minimal.parser.exit("\nSIGINT received")
        finally:
            intercom.print_final_averages()
    else:
        NAT_Traversal.NAT_STUN()
        NAT_Traversal.NAT_TYPE()
        quit()

## Ejecución

El código principal inicia la ejecución con las siguientes opciones:

- Si no se proporcionan, se ejecuta la función `NAT_STUN()` y se determina el tipo de NAT.
- Si se proporcionan puertos y direcciones personalizados, se ejecuta la comunicación de audio.
---


Ejecutamos NAT_traversal de manera local:

In [14]:
import os
os.chdir('/home/javier/Escritorio/NAT')
!python3 NAT_traversal.py

Puerto Externo: 54320, IP Externa: 217.216.183.232, End-Point: 217.216.183.232:54320, Tipo de NAT: Full Cone
Puerto Externo: 54320, IP Externa: 217.216.183.232, End-Point: 217.216.183.232:54320, Tipo de NAT: Full Cone
Puerto Externo: 54320, IP Externa: 217.216.183.232, End-Point: 217.216.183.232:54320, Tipo de NAT: Full Cone
Puerto Externo: 54320, IP Externa: 217.216.183.232, End-Point: 217.216.183.232:54320, Tipo de NAT: Full Cone
Puerto Externo: 54320, IP Externa: 217.216.183.232, End-Point: 217.216.183.232:54320, Tipo de NAT: Full Cone
Puerto Externo: 54320, IP Externa: 217.216.183.232, End-Point: 217.216.183.232:54320, Tipo de NAT: Full Cone
Puerto Externo: 54320, IP Externa: 217.216.183.232, End-Point: 217.216.183.232:54320, Tipo de NAT: Full Cone
Puerto Externo: None, IP Externa: None, End-Point: None:None, Tipo de NAT: Blocked
NAT abierto (Full Cone)
Puede establecer conexion con otro dispositivo. Para ello ejecute el siguiente comando: python3 NAT-traversal -l 'puerto de escuch

Intercambiamos datos y ejecutamos el siguiente comando para comunicarnos:

In [1]:
import os
os.chdir('/home/javier/Escritorio/NAT')
!python3 NAT_traversal.py -l 54320 -a 85.60.199.122 -p 54320

(INFO) minimal: A minimal InterCom (no compression, no quantization, no transform, ... only provides a bidirectional (full-duplex) transmission of raw (playable) chunks. 
(INFO) minimal: NUMBER_OF_CHANNELS = 2
(INFO) minimal: chunk_time = 0.023219954648526078 seconds
(INFO) minimal: Press enter-key to quit
^C

SIGINT received
